- MÅSTE TA BORT ID INNAN TRÄD CLASSIFIER

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import random

RANDOM_SEED = 12345

In [7]:
train_data = pd.read_csv("Dataset/train.csv")
#test_data = pd.read_csv("Dataset/test.csv")
#combined_data = [train_data,test_data]

X = train_data.drop('target', axis=1)
y = train_data.target

In [3]:
negative_class = (train_data.target == 0).sum()
positive_class = (train_data.target == 1).sum()
combined = negative_class+positive_class

print("Unbalanced ratio: ",positive_class/negative_class)
print("Positive class ratio: ", positive_class/combined)
print("Negative class ratio: ", negative_class/combined)

Unbalanced ratio:  0.037826188541597645
Positive class ratio:  0.036447517859182946
Negative class ratio:  0.963552482140817


We see that the positive class is 3.7 % the size of the negative class. We also see that the positive class corresponds to
3.6 % of the dataset.

Thus, the dataset is highly unbalanced.

Bagging:

In [4]:
#https://imbalanced-learn.org/stable/generated/imblearn.over_sampling.RandomOverSampler.html
ros = RandomOverSampler(random_state=RANDOM_SEED)
#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html
rus = RandomUnderSampler(random_state=RANDOM_SEED)

def unbalanced_bagger(M, x, y):
    #M = Amount of base_learners, x = train set, y = class labels
    base_learners = []
    sampling_techniques = [ros, rus]
    for i in range(M):
        #boostrap replica
        rnd = np.random.choice(x.shape[0], size=x.shape[0])
        x_bootstrap = x.iloc[rnd,:] #WHY?
        y_bootstrap = y.iloc[rnd]
        #sample the replica with random sample technique
        technique = random.randint(0,len(sampling_techniques)-1)
        x_sample, y_sample = sampling_techniques[technique].fit_resample(x_bootstrap,y_bootstrap)
        #train
        clf = DecisionTreeClassifier()
        clf.fit(x_sample,y_sample)
        base_learners.append(clf)

    def predict(x):
        predictions = np.zeros([x.shape[0], len(base_learners)])
        for i in range(len(base_learners)):
            predictions[:,i] = base_learners[i].predict(x)
        return np.sign(np.sum(predictions, axis=1))
    return predict

In [8]:
y[y == 0] = -1
X_train,X_test,y_train,y_test, = train_test_split(X,y, test_size=0.3)

<ipython-input-8-a9c66dd9ffbf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == 0] = -1


In [12]:
y_test[y_test == 0] = -1
ub = unbalanced_bagger(10,X_train,y_train)
testing_variable_delete = ub(X_test)
np.sum(ub(X_test) == y_test)/y_test.shape[0]

0.962092023028158